In [23]:
import numpy as np
import pandas as pd
import re
from sklearn import preprocessing
from scipy.sparse import csr_matrix
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs

In [4]:
sessions = {'HZKS0-WG8pZr0eCsZlBAP5Xm': ['INITIAL','login',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'home',
   'logout','$'],'5tPgZbHdK2Zp+heFBs8HsMkx': ['login',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'deferorder',
   'home',
   'logout']}

states = ["INITIAL","login","View_Items","home","logout","View_Items_quantity","Add_to_Cart","shoppingcart",
          "remove","deferorder","purchasecart","inventory","sellinventory","clearcart","cancelorder","$"]

In [137]:
def transition_matrix(sessions, states):
    
    for key, value in sessions.items():
        
        # labelEncoding
        le = preprocessing.LabelEncoder()
        le.fit(value)
        transformed_s = le.transform(value)
        #print(transformed_s)
        
        #factorize
        factorize = pd.factorize(value)[0]
        print(factorize)
        
        # matrix
        n = 1 + max(factorize)  # number of states
        M = [[0] * n for _ in range(n)]

        for (i, j) in zip(factorize, factorize[1:]):
            M[i][j] += 1
        
        # now convert to probabilities:
        for row in M:
            #print(row)
            s = sum(row)
            #print(s)
            if s > 0:
                row[:] = [f / s for f in row]

        for row in M: print(' '.join('{0:.2f}'.format(x) for x in row))
#test:
m = transition_matrix(sessions, states)
#for row in m: print(' '.join('{0:.2f}'.format(x) for x in row))


[0 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 4 5]
0.00 1.00 0.00 0.00 0.00 0.00
0.00 0.00 1.00 0.00 0.00 0.00
0.00 0.00 0.93 0.07 0.00 0.00
0.00 0.00 0.00 0.00 1.00 0.00
0.00 0.00 0.00 0.00 0.00 1.00
0.00 0.00 0.00 0.00 0.00 0.00
[0 1 2 1 2 1 2 1 2 1 2 3 4 3 4 3 4 3 4 5 6 7]
0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00
0.00 0.80 0.00 0.20 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00
0.00 0.00 0.00 0.75 0.00 0.25 0.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00


In [54]:
def transition_matrix(sessions, states):
    
    for key, value in sessions.items():
        
        # LabelEncoding
        le = preprocessing.LabelEncoder()
        le.fit(states)
        transformed_state = le.transform(states)
        print('LabelEncoding',transformed_state)
        print(list(le.inverse_transform(transformed_state)))
        
        # LabelEncoding
        le = preprocessing.LabelEncoder()
        le.fit(value)
        transformed_s = le.transform(value)
        print('LabelEncoding',transformed_s)
        print(list(le.inverse_transform(transformed_s)))
        
        factorize = pd.factorize(value)[0]
        #print("factorize",factorize)

        n = len(states)  # number of states
        M = [[0] * n for _ in range(n)]

        for (i, j) in zip(transformed_state, transformed_s[1:]):
            M[i][j] += 1
            #for row in M: print(' '.join('{0:.2f}'.format(x) for x in row))
            #print(M)
        # now convert to probabilities:
        for row in M:
            s = sum(row)
            if s > 0:
                row[:] = [f / s for f in row]
        
        #print(M)
        for row in M: print(' '.join('{0:.2f}'.format(x) for x in row))
#test:
m = transition_matrix(sessions, states)
#for row in m: print(' '.join('{0:.2f}'.format(x) for x in row))
#print(m)

LabelEncoding [ 2 10  3  8 11  4  1 15 13  7 12  9 14  6  5  0]
['INITIAL', 'login', 'View_Items', 'home', 'logout', 'View_Items_quantity', 'Add_to_Cart', 'shoppingcart', 'remove', 'deferorder', 'purchasecart', 'inventory', 'sellinventory', 'clearcart', 'cancelorder', '$']
LabelEncoding [1 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 5 0]
['INITIAL', 'login', 'View_Items', 'View_Items', 'View_Items', 'View_Items', 'View_Items', 'View_Items', 'View_Items', 'View_Items', 'View_Items', 'View_Items', 'View_Items', 'View_Items', 'View_Items', 'View_Items', 'home', 'logout', '$']
0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 1.00 0.00 0.00 0.00 0.

In [2]:
states = ["INITIAL","login","View_Items","home","logout","View_Items_quantity","Add_to_Cart","shoppingcart",
          "remove","deferorder","purchasecart","inventory","sellinventory","clearcart","cancelorder"]
sessions = {'HZKS0-WG8pZr0eCsZlBAP5Xm': ['INITIAL','login',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'home',
   'logout','$'],'5tPgZbHdK2Zp+heFBs8HsMkx': ['INITIAL','login',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'deferorder',
   'home',
   'logout','$']}

In [30]:
def transition_matrix(sessions, states):
    markovchains = []
    for key, value in sessions.items():
        # labelEncoding
        le = preprocessing.LabelEncoder()
        le.fit(value)
        transformed_s = le.transform(value)
        
        #factorize
        factorize = pd.factorize(value)[0]
        
        # matrix
        n = 1 + max(factorize)  # number of states
        M = [[0] * n for _ in range(n)]

        for (i, j) in zip(factorize, factorize[1:]):
            M[i][j] += 1
        
        # now convert to probabilities:
        for row in M:
            s = sum(row)
            if s > 0:
                row[:] = [f / s for f in row]
                
        # print Matrix style
        #for row in M: print(' '.join('{0:.2f}'.format(x) for x in row))
        
        # unique array in the right order
        value = np.array(value)
        _, idx = np.unique(value, return_index=True)
        
        df = pd.DataFrame(data = M, index=value[np.sort(idx)],
                          columns=value[np.sort(idx)])
        df_1 = pd.DataFrame(index=states, columns=states, dtype='float64')
        merge = df_1.merge(df, how='right').fillna(0).round(2).set_index(value[np.sort(idx)])
        
        # csr sparse matrix
        merge = csr_matrix(merge.values)
        markovchains.append([merge])
        
    return merge

        
#test:
m = transition_matrix(sessions, states)
m

<10x16 sparse matrix of type '<class 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [32]:
X = m
clustering = DBSCAN(eps=3, min_samples=2).fit(X)
clustering.labels_

clustering 

DBSCAN(algorithm='auto', eps=3, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=2, n_jobs=None, p=None)